# Add Wildcard and Null Block
Add special characters to the initial board configuration.

In [1]:
import itertools
import numpy as np
from IPython.display import clear_output
from sys import exit

## Utility Function


In [2]:
# check if a value is an integer
def is_int(value):
    try: int(value);   return True
    except: return False

In [3]:
class game(object):
    def __init__(self, w, h, n, init_board=None):
        self.player = 0
        self.status = 'active'

        # set up initial board
        if init_board is not None: 
            self.board = init_board
            w, h = init_board.shape
        else: self.board = np.array([[-1]*w]*h)
        self.w, self.h, self.n = w, h, n
        self.col_count = [next((h-k-1 for k in range(h)[::-1] if col[k] >-2), h) for col in self.board.T]
        self.feasible_moves = set(k for k in range(w) if self.col_count[k]<h)

        
    # display the status of the board
    def print_board(self):
        clear_output() # comment out for debugging
        for k in range(self.h):
            row = ' '.join(str(r) for r in self.board[k,:])
            row = row.replace('-3', '?').replace('-2', '#').replace('-1','.').replace('1','x')
            print('| ' + row + ' |')
        print('-'*(3+2*self.w))
        print('  ' + ' '.join(str(col) for col in range(self.w)))        

    # get user's input
    def col_input(self):
        # ask for and validate input
        while True:
            col = input('Player ' + str(self.player) + ': Enter the column to place your piece ' + 
                        '(' + str(self.player).replace('1','x') + ') ' )
            if col.lower()=='q': print('Thanks for playing!');   exit(0)
            if col=='`': col = 0 # define an alias for 0

            # check for valid input
            if is_int(col): 
                col = int(col)
                if col>=0 and col<self.w and col in self.feasible_moves: break
        return col

    # count connections to the left and right of pos
    def cnct_gt_n(self, vec, pos): 
        count, player = 1, vec[pos]
        
        # check if at least n in length
        if sum(np.logical_or(vec==player, vec==-3)) < self.n: return False

        for v in vec[:pos][::-1]:
            if v==player or v==-3: count += 1
            else: break
        for v in vec[pos+1:]:
            if v==player or v==-3: count += 1
            else: break
        return count >= self.n

    # extract backward diagonal from a nested list
    def bdiag(self, board, shift=0):
        if shift==0: return [board[k, self.w-k-1] for k in range(min(self.h, self.w))]
        if shift>0: return [board[k, self.w-(k+shift)-1] for k in range(min(self.h, self.w-shift))]
        return [board[k-shift, self.w-k-1] for k in range(min(self.h+shift, self.w))]   

    # check intersecting row, col, diagonals for connect-n
    def connect_n_check(self, row, col):
        forward_diag = np.diag(self.board, col-row)
        backward_diag = self.bdiag(self.board, self.w-1-col-row)
        return (self.cnct_gt_n(self.board[:, col], row) or             # col
                self.cnct_gt_n(self.board[row, :], col) or             # row
                self.cnct_gt_n(forward_diag, min(row, col)) or         # fdiag
                self.cnct_gt_n(backward_diag, min(row, self.w-1-col))) # bdiag

    # update game        
    def update_game(self, col):
        # update col_count
        self.col_count[col] += 1
        
        # skip special pieces
        row = self.h-self.col_count[col]
        while self.board[row, col]<-1: 
            row -= 1 
            self.col_count[col] += 1
            
        # update board, player and feasible moves
        self.board[row, col] = self.player # assign move to player
        self.player = 1*(self.player==0)   # switch players
        if self.col_count[col]==self.h: self.feasible_moves.remove(col)
                    
        # update status
        if self.connect_n_check(row, col): self.status = 'won';   return
        if not self.feasible_moves: self.status = 'draw'
        
    # two-player game
    def play_game(self):
        while True:
            self.print_board()
            col = self.col_input()
            self.update_game(col)
            if self.status!='active':
                self.print_board()
                if self.status=='won': print('Player ' + str(1-self.player) + ' won!')
                else: print('Game is a draw.')
                return            

In [4]:
w, h, n = 4, 4, 4
init_board = np.array([[-1]*w]*h)
init_board[:,2]= -3

g = game(w, h, n, init_board)
g.play_game()

| 0 x ? . |
| 0 0 ? x |
| x x ? 0 |
| x 0 ? 0 |
-----------
  0 1 2 3
Player 0 won!
